In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from math import sqrt

pd.options.mode.chained_assignment = None

In [30]:
#ibm_df = pd.read_csv("raw_data/3Stocks/IBM_1min.csv")
#jnj_df = pd.read_csv("raw_data/3Stocks/JNJ_1min.csv")
wmt_df = pd.read_csv("raw_data/3Stocks/WMT_1min.csv")

In [33]:
df['group'] = (df.index // 381)

# Step 2: Calculate the sum of volumes for each group
group_sums = df.groupby('group')['Volume'].transform('sum')

# Step 3: Add the group sums as a new column
df['group_volume_sum'] = group_sums

# Optional: If you don't need the 'group' column anymore, you can drop it
df.drop(columns=['group'], inplace=True)

In [36]:
df.iloc[:382]

,Date Time,Open,High,Low,Close,Volume,Date,Time,group_volume_sum
0,2010-01-04 09:35:00,40.5048,40.5876,40.4973,40.5876,105991.0,2010-01-04,09:35:00,16965492.0
1,2010-01-04 09:36:00,40.5876,40.6102,40.5801,40.5876,41443.0,2010-01-04,09:36:00,16965492.0
2,2010-01-04 09:37:00,40.5876,40.6177,40.5801,40.5876,46632.0,2010-01-04,09:37:00,16965492.0
3,2010-01-04 09:38:00,40.5876,40.5876,40.5424,40.5424,45112.0,2010-01-04,09:38:00,16965492.0
4,2010-01-04 09:39:00,40.5349,40.5650,40.5349,40.5462,40718.0,2010-01-04,09:39:00,16965492.0
...,...,...,...,...,...,...,...,...,...
377,2010-01-04 15:52:00,40.8510,40.8585,40.8434,40.8434,118598.0,2010-01-04,15:52:00,16965492.0
378,2010-01-04 15:53:00,40.8434,40.8510,40.8359,40.8359,71667.0,2010-01-04,15:53:00,16965492.0
379,2010-01-04 15:54:00,40.8359,40.8510,40.8209,40.8510,134105.0,2010-01-04,15:54:00,16965492.0
380,2010-01-04 15:55:00,40.8434,40.8510,40.8434,40.8434,75663.0,2010-01-04,15:55:00,16965492.0


In [32]:
df = wmt_df

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# import os

# os.chdir allows you to change directories, like cd in the Terminal
# os.chdir('/content/drive/MyDrive/Colab Notebooks/data-cifar-classification')

In [5]:
# (images_train, labels_train), (images_test, labels_test)

In [6]:
# (50000, 32, 32, 3)

In [25]:
ibm_df

NameError: name 'ibm_df' is not defined

In [38]:
def preprocess_dataframe(df, split_size = 0.7):

    df["Date Time"] = pd.to_datetime(df["Date Time"])
    df["Log_Returns"] = np.log(df["Close"] / df["Close"].shift(1))
    
    #df["log_return_squared"] = df["Log_Returns"].apply(lambda x: x**2)
    
    #df["realized_var"] = df["log_return_squared"].cumsum()
    
    #df["realized_vol"] = df["realized_var"].apply(lambda x: sqrt(x))
    #df["forward_RV"] = df["realized_vol"].shift(1)

    # Create a flag to group by 381 row days
    df["day"] = (df.index // 381)

    # Calculate the RV for each grouped 381 row day
    group_sums = df.groupby("day")["Log_Returns"].transform(lambda x: np.sqrt(np.sum(x**2)))
    
    # Add the group sums as a new column
    df["realized_vol"] = group_sums


    df = df.iloc[381:]
    #df.reset_index(drop=True, inplace=True)
    df.set_index("Date Time", inplace=True)
    
    split_index = int(np.floor(df.shape[0] * split_size / 381) * 381)

    df_train = df.iloc[:split_index]
    df_test = df.iloc[split_index:]

    return df_train, df_test

In [41]:
wmt_df

,Date Time,Open,High,Low,Close,Volume,Date,Time,group_volume_sum,Log_Returns,day,realized_vol
0,2010-01-04 09:35:00,40.5048,40.5876,40.4973,40.5876,105991.0,2010-01-04,09:35:00,16965492.0,NaN,0,0.009153
1,2010-01-04 09:36:00,40.5876,40.6102,40.5801,40.5876,41443.0,2010-01-04,09:36:00,16965492.0,0.000000,0,0.009153
2,2010-01-04 09:37:00,40.5876,40.6177,40.5801,40.5876,46632.0,2010-01-04,09:37:00,16965492.0,0.000000,0,0.009153
3,2010-01-04 09:38:00,40.5876,40.5876,40.5424,40.5424,45112.0,2010-01-04,09:38:00,16965492.0,-0.001114,0,0.009153
4,2010-01-04 09:39:00,40.5349,40.5650,40.5349,40.5462,40718.0,2010-01-04,09:39:00,16965492.0,0.000094,0,0.009153
...,...,...,...,...,...,...,...,...,...,...,...,...
958591,2019-12-31 15:51:00,114.5096,114.5289,114.4903,114.4903,20873.0,2019-12-31,15:51:00,2482293.0,-0.000169,2515,0.006117
958592,2019-12-31 15:52:00,114.4807,114.5289,114.4807,114.5193,24827.0,2019-12-31,15:52:00,2482293.0,0.000253,2515,0.006117
958593,2019-12-31 15:53:00,114.5193,114.5241,114.5096,114.5096,23027.0,2019-12-31,15:53:00,2482293.0,-0.000085,2515,0.006117
958594,2019-12-31 15:54:00,114.5144,114.5771,114.5096,114.5675,21725.0,2019-12-31,15:54:00,2482293.0,0.000506,2515,0.006117


In [42]:
tmp = create_subsequences(wmt_df)

last iteration


In [57]:
wmt_df["realized_vol"][381*27]

0.008763196505731576

In [44]:
tmp[1]

[0.009369260049674014,
 0.012099991860970228,
 0.01247066419680605,
 0.008501454280680925,
 0.010780124168609224,
 0.009960463514853277,
 0.008763196505731576,
 0.00963241235859543,
 0.006531777248365041,
 0.010262160160824058,
 0.02222972381909325,
 0.009169500175437698,
 0.006444545364548057,
 0.009333656818953715,
 0.006925319289449158,
 0.01171275496709077,
 0.009714967306401388,
 0.01085900203595727,
 0.00658781468735642,
 0.00763823381868501,
 0.008529075351059552,
 0.0053033112079996165,
 0.004895150955327444,
 0.008093848610889685,
 0.007863551095672196,
 0.006288021056026307,
 0.007983695328830705,
 0.01580359094530811,
 0.01065189619422148,
 0.0075662971925327975,
 0.006374717795739341,
 0.007155637223747634,
 0.0092521951820511,
 0.007045029800131986,
 0.006643843193232634,
 0.0075292413281903055,
 0.006431071622795864,
 0.006101689779589982,
 0.007012893451269073,
 0.006950617432005171,
 0.008601231158113972,
 0.0069723738660494835,
 0.006305134673910544,
 0.008746343452873

In [8]:
def create_subsequences(dataframe):

    # contains len(ibm_train_split)/381 - 20 = 1741 
    # dataframes with 8001 rows meaning 21 days with 381 hours
    sequence_list = []
    sequence_target = []
    
    for i in range(int(len(dataframe)/381-20)):
        tmp_subsequence = dataframe.iloc[i * 381: (21 + i) * 381]
        sequence_list.append(tmp_subsequence)

        try:
            subsequence_target = dataframe.iloc[(21 + i) * 381]["realized_vol"]
            sequence_target.append(subsequence_target)
        except:
            print("last iteration")

    return sequence_list, sequence_target

In [9]:
def month_to_image(one_month):

    size = len(one_month)

    # creating (8001,) sized arrays
    red = np.zeros(size)
    green = np.zeros(size)
    blue = np.zeros(size)

    # adding negative returns absolute values to red channel
    # and positive returns to green channel
    # leave blue channel at zeros
    for i in range(len(one_month)):
        log_return = one_month[i]

        if log_return < 0:
            red[i] = abs(log_return)
        elif log_return > 0:
            green[i] = log_return
        elif log_return == 0:
            continue

    # turning shape (8001,) into (8001,1) for scaling
    red = np.reshape(np.array(red), (-1,1))
    green = np.reshape(np.array(green), (-1,1))

    # scaling returns to [0, 255] interval
    mm_scaler_red = MinMaxScaler(feature_range=(0,255))
    red_scaled = mm_scaler_red.fit_transform(red)
    
    mm_scaler_green = MinMaxScaler(feature_range=(0,255))
    green_scaled = mm_scaler_green.fit_transform(green)

    # flatten (8001, 1) arrays into (8001,) shape to stack them
    red_scaled_flat = red_scaled.flatten()
    green_scaled_flat = green_scaled.flatten()

    # stacking three (8001,) arrrays into one (8001, 3)
    flat_image = np.column_stack((red_scaled_flat,
                                  green_scaled_flat,
                                  blue))

    # reshaping (8001, 3) array into (21, 381, 3)
    # for 3 rgb values for 381 hours for 21 days
    square_image = flat_image.reshape((21, 381, 3))

    return square_image

In [10]:
def create_images(sequence_list):

    image_list = []
    
    for rolling_month in sequence_list:

        X_array = np.array(rolling_month["Log_Returns"])
        
        image = month_to_image(X_array)
        
        image_list.append(image)

    return image_list

In [11]:
def get_subsequence_images(dataframe):

    month_sequences = create_subsequences(dataframe)

    image_list = create_images(month_sequences[0])

    return image_list, np.array(month_sequences[1])

In [21]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Input

def initialize_model():
    
    model = Sequential()

    model.add(Input(shape=(21, 381, 3))) 
    
    model.add(Conv2D(16, (3, 3), activation = 'relu', padding = 'same'))#, input_shape=(21, 381, 3)))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Dropout(0.2))
    
    model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (2, 2), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(100, activation = 'relu'))
    model.add(Dense(50, activation = 'relu'))
    #model.add(Dropout(0.7))
    model.add(Dense(1, activation = 'linear'))
    
    return model

In [13]:
from tensorflow.keras import optimizers

def compile_model(model):
    model.compile(loss = 'mse',
                  optimizer = 'adam')
    return model

In [14]:
from tensorflow.keras.callbacks import EarlyStopping


def prepare_and_train_cnn(df, epochs = 100, patience = 10):

    train, test = preprocess_dataframe(df, split_size = 0.7)

    X_train, y_train = get_subsequence_images(train)
    X_test, y_test = get_subsequence_images(test)

    del X_test[-1]
    del X_train[-1]

    X_train_arr = np.array(X_train)
    X_test_arr = np.array(X_test)
    
    X_train_norm = X_train_arr / 255.
    X_test_norm = X_test_arr / 255.



    model = initialize_model()
    model = compile_model(model)

    es = EarlyStopping(patience = patience, verbose = 2)

    history = model.fit(X_train_norm, y_train, 
                    validation_split = 0.2,
                    callbacks = [es], 
                    epochs = epochs, 
                    batch_size = 16)

    evaluate = model.evaluate(X_test_norm, y_test, verbose = 2)
    
    return model, history, evaluate

In [15]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label = 'train' + exp_name)
    ax1.plot(history.history['val_loss'], label = 'val' + exp_name)
    ax1.set_ylim(0., 0.15)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['mae'], label='train mae'  + exp_name)
    ax2.plot(history.history['val_mae'], label='val mae'  + exp_name)
    ax2.set_ylim(0, 0.35)
    ax2.set_title('MAE')
    ax2.legend()
    return (ax1, ax2)

In [39]:
wmt = prepare_and_train_cnn(wmt_df)

last iteration
last iteration
Epoch 1/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0011 - val_loss: 6.9515e-05
Epoch 2/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 1.5698e-05 - val_loss: 7.4862e-05
Epoch 3/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 1.4032e-05 - val_loss: 7.1640e-05
Epoch 4/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 2.0283e-05 - val_loss: 6.1461e-05
Epoch 5/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 1.7650e-05 - val_loss: 6.1803e-05
Epoch 6/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 1.4383e-05 - val_loss: 6.6293e-05
Epoch 7/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 2.0593e-05 - val_loss: 5.8508e-05
Epoch 8/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 1.2990e-05 - val_loss: 6.0443e-05
Epoch 9/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 1.9420e-05 - val_loss: 6.1630e-05
Epoch 10/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 1.3809e-05 - val_loss: 6.4903e-05
Epoch 11/100
87/87 ━━━━━━━━━━━━━━

In [40]:
wmt[0].summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 21, 381, 16)    │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 10, 190, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 10, 190, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 10, 190, 32)    │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 5, 95, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 5, 95, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 5, 95, 64)      │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 2, 47, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 2, 47, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 6016)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 100)            │       601,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,860,437 (7.10 MB)

 Trainable params: 620,145 (2.37 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,240,292 (4.73 MB)

7.53e-05

In [62]:
print(format(0.00007530, '.8f'))

0.00007530


In [61]:
print(format(wmt[2], '.8f'))

0.00005732


In [58]:
wmt_df

,Date Time,Open,High,Low,Close,Volume,Date,Time,group_volume_sum,Log_Returns,day,realized_vol
0,2010-01-04 09:35:00,40.5048,40.5876,40.4973,40.5876,105991.0,2010-01-04,09:35:00,16965492.0,NaN,0,0.009153
1,2010-01-04 09:36:00,40.5876,40.6102,40.5801,40.5876,41443.0,2010-01-04,09:36:00,16965492.0,0.000000,0,0.009153
2,2010-01-04 09:37:00,40.5876,40.6177,40.5801,40.5876,46632.0,2010-01-04,09:37:00,16965492.0,0.000000,0,0.009153
3,2010-01-04 09:38:00,40.5876,40.5876,40.5424,40.5424,45112.0,2010-01-04,09:38:00,16965492.0,-0.001114,0,0.009153
4,2010-01-04 09:39:00,40.5349,40.5650,40.5349,40.5462,40718.0,2010-01-04,09:39:00,16965492.0,0.000094,0,0.009153
...,...,...,...,...,...,...,...,...,...,...,...,...
958591,2019-12-31 15:51:00,114.5096,114.5289,114.4903,114.4903,20873.0,2019-12-31,15:51:00,2482293.0,-0.000169,2515,0.006117
958592,2019-12-31 15:52:00,114.4807,114.5289,114.4807,114.5193,24827.0,2019-12-31,15:52:00,2482293.0,0.000253,2515,0.006117
958593,2019-12-31 15:53:00,114.5193,114.5241,114.5096,114.5096,23027.0,2019-12-31,15:53:00,2482293.0,-0.000085,2515,0.006117
958594,2019-12-31 15:54:00,114.5144,114.5771,114.5096,114.5675,21725.0,2019-12-31,15:54:00,2482293.0,0.000506,2515,0.006117
